In [ ]:
from openai import OpenAI
client = OpenAI(api_key="OPENAI_API_KEY")

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは自治体の職員です"},
        {"role": "user", "content": "ごみ収集日は？"}
    ]
)

print(response.choices[0].message.content)


In [3]:
faq = [
    "可燃ごみは毎週月曜日と木曜日に回収します。",
    "粗大ごみは事前予約制です。",
    "街灯の故障は道路課にご連絡ください。",
    "住民票は市役所1階で発行できます。",
    "■所得証明○普通徴収の場合（事業所得者の方など）　毎年６月１日○給与からの特別徴収の場合毎年５月下旬○公的年金からの特別徴収の場合毎年６月１日",
    "勤労学生控除の適用を受けるためには、市民税・県民税の申告が必要です",
    "愛知県内の病院などにかかったときにマイナ保険証等とともに医療証を提示していただきますと、保険診療の自己負担額を名古屋市が負担します。",
    " 各種証明書の発行には本人確認書類が必要です。",
    "敬老パスは更新前の有効期限の翌日から1年間有効になります。",
    "固定資産税は毎年4期に分けて納付します。",
    "市民税の納付書は6月に発送されます。",
    "口座振替の申し込みが可能です。",


]


In [4]:
emb = client.embeddings.create(
    model="text-embedding-3-small",
    input=faq
)

vectors = [e.embedding for e in emb.data]


In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

query = "敬老パスについて知りたい。"

query_vec = client.embeddings.create(
    model="text-embedding-3-small",
    input=query
).data[0].embedding

scores = [cosine_similarity(query_vec, v) for v in vectors]

if max(scores) < 0.7:
    print("該当する情報が見つかりませんでした。")




best_index = np.argmax(scores)
print(faq[best_index])


In [ ]:
top_indices = np.argsort(scores)[-3:][::-1]
contexts = [faq[i] for i in top_indices]
context = "\n\n".join(contexts)


prompt = f"""
あなたは自治体の職員です。
必ず以下のFAQの情報のみを根拠に回答してください。
FAQに記載がない内容は推測せず、
「担当課にお問い合わせください」と回答してください。

FAQ:
{context}

質問:
{query}
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
